In [5]:
import numpy as np
import time

import pickle
import mne
import eegraph
print(mne.__version__)

import gradio as gr
print(gr.__version__) 

1.0.3
3.0.24


In [10]:
    
def graph_gen(file_obj,measure):
    global G,graphs,connectivity_matrix,n_graphs
    raw_fname=file_obj.name
    G = eegraph.Graph()
    G.load_data(path = raw_fname,electrode_montage_path = '1.ced')
    if measure=="pearson_correlation":   
        graphs, connectivity_matrix = G.modelate(window_size = 2, connectivity = 'pearson_correlation')#threshold = 0.8
    elif measure=="squared_coherence":
        graphs, connectivity_matrix = G.modelate(window_size = 2, connectivity = 'squared_coherence',bands = ['delta','theta','alpha'])
    n_graphs=len(graphs)
    framenum=len(connectivity_matrix)
    A1=[]
    for n in range(framenum):
        A=connectivity_matrix[n,:,:] 
        A=A.flatten()
        A1.append(A)
    A1 = np.array(A1) 
    with open('FunctionalConnectivitymodel.pickle','rb') as fr:
       graphclassificationmodel = pickle.load(fr)
       predicted_labels=graphclassificationmodel.predict(A1)   
    mask = np.unique(predicted_labels)
    tmp = []
    for v in mask:
       tmp.append(np.sum(predicted_labels==v))
       ts = np.max(tmp)
       Total=predicted_labels.size
       ts1=Total-ts
       max_v = mask[np.argmax(tmp)]
       Prob=ts/Total*100
       Prob1=ts1/Total*100
       if max_v==1:
           pred='Responder'
       else:
           pred='Non-Responder'
    return (f'{round(Prob,2)}% chance this person is {pred}'),(f'{n_graphs} connectivity graphs generated')

    
def visualise(number): 
    grph_number=number-1
    G.visualize(graphs[grph_number], 'graph_1')
    path='graph_1_plot.html'
    return path
    

demo = gr.Blocks()

with demo:
    gr.Image('neuroinformatics_logo.png',image_mode='L',label='AI in Tinnitus').style(height=54, width=240)
    gr.Markdown(
    """
    # Tinnitus Prediction using Functional Connectivity
    """)
    gr.Markdown(
    """
    Upload the eeg data to see the prediction.
    
    """)
    inp1 = gr.File(label="Upload eeg data")
    inp2 = gr.Radio(["pearson_correlation", "squared_coherence"],label="Select a connectivity measure",value="pearson_correlation")
    btn1 = gr.Button("Compute Functional Connectivity and Make the Prediction",variant='primary')
    out1 = gr.Label(label="Predicted Results")
    lab1 = gr.Label(label="Graphs Generated")
    btn1.click(fn=graph_gen, inputs=[inp1,inp2], outputs=[out1,lab1])
    gr.Markdown(
    """
  ***********************************************************************************************
  ***********************************************************************************************
  
    """)
    gr.Markdown(
    """
    # Visualise the Connectivity Graph
    """)
    gr.Markdown(
    """
    Enter any number less than or equal to 'the total number of Graphs Generated'.
    
    """)
    
    inp3 = gr.Number(label="Specify the graph number you want to visualize",value=1)
    btn2 = gr.Button("Visualise Connectivity Graphs",variant='primary')
    out2 = gr.File()
    btn2.click(fn=visualise, inputs=inp3, outputs=out2)
       
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7866/
Running on public URL: https://16786.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x133ee87f0>,
 'http://127.0.0.1:7866/',
 'https://16786.gradio.app')

Extracting EDF parameters from /var/folders/ts/975gflv16y9_wd421n6z6w8c0000gn/T/Noel R Pre-Decitw2g4d2y.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

EEG Information.
Number of Channels: 72
Sample rate: 256.0 Hz.
Duration: 302.996 seconds.
Channel Names: ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 512.0), (512.0, 1024.0), (1024.0, 1536.0), (1536.0, 2048.0), (2048.0, 2560.0), (2560.0, 3072.0), (3072.0, 3584.0), (3584.0, 4096.0), (4096.0, 4608.0), (4608.0

/Users/manjuvallayil/miniforge3/envs/tinnitus/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/manjuvallayil/miniforge3/envs/tinnitus/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/manjuvallayil/miniforge3/envs/tinnitus/eegraph/tools.py:647: UserWarning: Channel with label 'Iz' is not a recognized electrode position. It